In [5]:
import pandas as pd  
import numpy as np
import boto3
import tempfile
import os

In [6]:
#Load some data
s3 = boto3.resource('s3', region_name='us-east-1')
bucket = s3.Bucket('song-feature-csvs')

#open the essentia file. This downloads the file locally to the instance
#so that it doesnt have to fit directly into memory
fname = 'essentia_combined_csv_500rows.csv'
if os.path.isfile(fname) == False:
    %sc
    !wget 'https://song-feature-csvs.s3.amazonaws.com/essentia_combined_csv_500rows.csv'
    
df_essentia = pd.read_csv('essentia_combined_csv_500rows.csv'); 

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3596,7081,7082,7083) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
fname = 'features.csv'
if os.path.isfile(fname) == False:
    %sc
    !wget 'https://song-feature-csvs.s3.amazonaws.com/features.csv'
    
preexisting_features = pd.read_csv('features.csv', header=[0,1]); 

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
fname = 'raw_tracks.csv'
if os.path.isfile(fname) == False:
    %sc
    !wget 'https://song-feature-csvs.s3.amazonaws.com/raw_tracks.csv' 
    
preexisting_tracks = pd.read_csv(fname); 

In [9]:
#adjust unique IDs and cleanup column names
df_essentia['song_id_prefix'] = df_essentia['song_id'].str[:6]
preexisting_features.columns = preexisting_features.columns.map('_'.join)
preexisting_features = preexisting_features.drop([0,1])
preexisting_features.rename(columns={'feature_statistics':'song_id_prefix'}, inplace=True)
preexisting_features['song_id_prefix'] = preexisting_features['song_id_prefix'].apply('{:0>6}'.format)
preexisting_tracks['song_id_prefix'] = preexisting_tracks['track_id'].apply('{:0>6}'.format)

In [10]:
#Lets find nulls
df_essentia.describe()

,lowlevel_average_loudness,lowlevel_barkbands_crest_dmean,lowlevel_barkbands_crest_dmean2,lowlevel_barkbands_crest_dvar,lowlevel_barkbands_crest_dvar2,lowlevel_barkbands_crest_max,lowlevel_barkbands_crest_mean,lowlevel_barkbands_crest_median,lowlevel_barkbands_crest_min,lowlevel_barkbands_crest_stdev,...,rhythm_beats_position_2672,rhythm_beats_position_2673,rhythm_beats_position_2674,rhythm_beats_position_2675,rhythm_beats_position_2676,rhythm_beats_position_2677,rhythm_beats_position_2678,rhythm_beats_position_2679,metadata_tags_length_0,metadata_tags_bpm_0
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,2.00000,1.0
mean,0.649411,2.585637,4.208796,6.138574,15.568752,25.905858,11.104531,10.332064,2.654378,4.439218,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,331393.50000,0.0
std,0.326733,0.499449,0.852130,2.010181,4.958697,1.090803,2.262386,2.494333,0.545209,0.766630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89840.03787,NaN
min,0.000017,1.303844,1.992265,1.842394,4.282618,21.746363,5.927392,5.334275,1.557132,2.317838,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,267867.00000,0.0
25%,0.398164,2.237773,3.643707,4.651996,12.107816,25.458133,9.579603,8.652435,2.262041,3.907986,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,299630.25000,0.0
50%,0.794821,2.577520,4.218106,6.037499,15.294977,26.325953,10.866168,9.967081,2.553303,4.414587,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,331393.50000,0.0
75%,0.928405,2.909029,4.727415,7.455754,18.783432,26.701862,12.515655,11.858310,2.975021,4.988098,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,363156.75000,0.0
max,0.986739,4.636729,7.696652,14.694959,34.904377,26.999681,20.685989,22.121281,5.321369,7.125858,...,1196.234009,1196.686768,1197.12793,1197.569092,1198.010376,1198.463135,1198.904297,1199.345459,394920.00000,0.0


In [11]:
preexisting_features.describe()

,chroma_cens_kurtosis,chroma_cens_kurtosis.1,chroma_cens_kurtosis.2,chroma_cens_kurtosis.3,chroma_cens_kurtosis.4,chroma_cens_kurtosis.5,chroma_cens_kurtosis.6,chroma_cens_kurtosis.7,chroma_cens_kurtosis.8,chroma_cens_kurtosis.9,...,tonnetz_std.3,tonnetz_std.4,tonnetz_std.5,zcr_kurtosis,zcr_max,zcr_mean,zcr_median,zcr_min,zcr_skew,zcr_std
count,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,...,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000,106574.000000
mean,0.439999,0.199756,0.334683,0.270263,0.294806,0.315635,0.273511,0.334927,0.201939,0.285631,...,0.098667,0.021569,0.021880,29.464638,0.381368,0.052755,0.043576,0.002342,3.214247,0.040303
std,32.285524,11.120903,9.980692,19.074557,15.025603,12.176182,16.975991,15.486402,7.364308,8.175257,...,0.037101,0.006076,0.006125,88.975395,0.183414,0.032634,0.031703,0.005152,3.045345,0.027259
min,-1.955806,-1.936607,-1.967450,-3.000000,-1.921206,-1.923740,-1.935382,-3.000000,-3.000000,-3.000000,...,0.005681,0.000087,0.000368,-1.967539,0.010254,0.000055,0.000000,0.000000,-15.727972,0.000436
25%,-0.636597,-0.665684,-0.629114,-0.644747,-0.656779,-0.622579,-0.646798,-0.635741,-0.667799,-0.660292,...,0.071240,0.017226,0.017542,3.316640,0.246582,0.032913,0.024902,0.000000,1.400700,0.022054
50%,-0.176557,-0.221131,-0.152708,-0.191032,-0.198025,-0.152089,-0.187959,-0.161478,-0.214547,-0.181836,...,0.093500,0.020636,0.020964,9.714652,0.357910,0.047077,0.037598,0.000000,2.404834,0.033566
75%,0.485000,0.410097,0.529836,0.446800,0.452680,0.513112,0.472161,0.525814,0.432480,0.495568,...,0.121007,0.024990,0.025252,26.981386,0.471680,0.064616,0.054199,0.002930,4.079428,0.051275
max,8843.367188,2864.075195,1777.615967,4257.045898,3188.034424,2809.779297,4281.090820,3869.482178,1217.209473,1772.918213,...,0.332470,0.092869,0.116128,8438.014648,0.999512,0.878024,0.895996,0.262207,78.084976,0.424317


In [12]:
#combine all datasets
result = pd.merge(df_essentia, preexisting_features, how='inner', on=['song_id_prefix', 'song_id_prefix'])
result = pd.merge(result, preexisting_tracks, how='inner', on=['song_id_prefix', 'song_id_prefix'])

#add a genre column and extract it
for index, row in result.iterrows():
    try:
        genre = eval(str(row['track_genres']))[0]['genre_title']
    except:
        genre = row['track_genres']   
    result['genre'] = genre

result.drop(['song_id','metadata_audio_properties_analysis_downmix', 
         'metadata_audio_properties_analysis_equal_loudness', 
         'metadata_audio_properties_analysis_length', 'metadata_audio_properties_analysis_sample_rate', 
         'metadata_audio_properties_analysis_start_time', 'metadata_audio_properties_bit_rate', 
         'metadata_audio_properties_codec', 'metadata_audio_properties_length', 
         'metadata_audio_properties_lossless', 'metadata_audio_properties_md5_encoded', 
         'metadata_audio_properties_number_channels', 'metadata_audio_properties_replay_gain', 
         'metadata_audio_properties_sample_rate', 'metadata_tags_album_0', 
         'metadata_tags_albumartist_0', 'metadata_tags_artist_0', 
         'metadata_tags_composer_0', 'metadata_tags_contentgroup_0', 
         'metadata_tags_copyright_0', 'metadata_tags_date_0', 
         'metadata_tags_date_1', 'metadata_tags_discnumber_0', 
         'metadata_tags_encodedby_0', 'metadata_tags_file_name', 
         'metadata_tags_filetype_0', 'metadata_tags_title_0', 'metadata_tags_tracknumber_0', 
         'metadata_version_essentia', 'metadata_version_essentia_git_sha', 'metadata_version_extractor'],axis=1)

result = result.dropna(axis='columns')

In [13]:
print(result.isnull().sum())

song_id                                  0
lowlevel_average_loudness                0
lowlevel_barkbands_crest_dmean           0
lowlevel_barkbands_crest_dmean2          0
lowlevel_barkbands_crest_dvar            0
lowlevel_barkbands_crest_dvar2           0
lowlevel_barkbands_crest_max             0
lowlevel_barkbands_crest_mean            0
lowlevel_barkbands_crest_median          0
lowlevel_barkbands_crest_min             0
lowlevel_barkbands_crest_stdev           0
lowlevel_barkbands_crest_var             0
lowlevel_barkbands_flatness_db_dmean     0
lowlevel_barkbands_flatness_db_dmean2    0
lowlevel_barkbands_flatness_db_dvar      0
lowlevel_barkbands_flatness_db_dvar2     0
lowlevel_barkbands_flatness_db_max       0
lowlevel_barkbands_flatness_db_mean      0
lowlevel_barkbands_flatness_db_median    0
lowlevel_barkbands_flatness_db_min       0
lowlevel_barkbands_flatness_db_stdev     0
lowlevel_barkbands_flatness_db_var       0
lowlevel_barkbands_kurtosis_dmean        0
lowlevel_ba

In [ ]:
input_dim = result.shape[1]-1

In [ ]:
import tensorflow.keras as keras 
from keras.constraints import maxnorm
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    activation='relu'
    dropout_rate=0.0
    init_mode='uniform'
    weight_constraint=0
    optimizer='adam'
    lr=0.01
    momentum=0
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=input_dim, kernel_initializer=init_mode,
                    activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7

# split into input (X) and output (Y) variables
X = result.loc[:, result.columns != 'genre']
Y = result['genre']

#Split the data for RNN classification

lstm_train_x, lstm_test_x, lstm_train_y, lstm_test_y = train_test_split(X, Y, test_size=.25)
#lstm_train_x, lstm_validation_x, lstm_train_y, lstm_validation_y = train_test_split(lstm_train_x, lstm_train_y, test_size=.2)


# create model
model = KerasClassifier(build_fn=create_model, batch_size=1000, epochs=10)

#gridsearch
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
dropout_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
weight_constraint = [1, 2, 3, 4, 5]
neurons = [1, 5, 10, 15, 20, 25, 30]
init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

epochs = [1, 10, 50, 100]
betch_size = [1000, 5000]
param_grid = dict(epochs=epochs, batch_size=batch_size)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(lstm_train_x, lstm_train_y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))